In [ ]:
import requests
import csv
import json
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import io
import seaborn as sns
import numpy as np
from copy import copy
from Yash.CALFEWS_Files.scraper import Scraper

In [ ]:
start_string = '1995-10-01'
dsf = "%Y-%m-%d"
# data types that are common to all basins/reservoirs
labels_basin = ['snow', 'fnf', 'storage', 'inf', 'otf', 'evap', 'precip', 'gains', 'fci']
# these are the keys we will associate with each basin/reservoir


In [ ]:
# some CDEC station names are different for some data types,
# final data will all be associated with these keys
#TRT is the added key for Trinity
stations_basin = ['SHA', 'TRT', 'ORO', 'YRS', 'FOL', 'MHB', 'PAR', 'NHG', 'NML', 'DNP', 'EXC', 'MIL', 'PFT', 'KWH', 'SUC', 'ISB']


In [ ]:
# this initializes the object that will become the input scenario
calfews_data = Scraper(stations_basin, labels_basin, timestep = 'd', start_date = start_string, date_string_format = dsf)


In [ ]:
#Checks
calfews_data.data_timeseries

In [ ]:
# monthly inputs (extend further back)
start_string_monthly = '1905-10-01'
labels_basin_monthly = ['fnf', 'inf']
calfews_data_m = Scraper(stations_basin, labels_basin_monthly, timestep = 'm', start_date = start_string_monthly, date_string_format = dsf)


In [ ]:
#Checks
calfews_data_m.data_timeseries

In [ ]:
# some additional data does not conform with the above basin/data type pairs above
#Generate data columns for San Luis, Delta Pumps, and Old-Middle River Flows
san_luis_labels = ['SL', 'SLF', 'SLS', 'TRP', 'HRO', 'OMR']
san_luis_suffix = ['storage', 'storage', 'storage', 'pump', 'pump', '']
calfews_data.add_extra_stations(san_luis_labels, san_luis_suffix)


In [ ]:
#Checks
calfews_data.data_timeseries

In [ ]:
# Generate data columns for incremental flow to Rio Vista, Vernalis, delta inflow from eastside streams, 
# in-delta depletions, pumping into contra-costa canal and marker slough, total delta inflow, total vernalis flow
delta_labels = ['SAC', 'SJ', 'EAST', 'delta', 'CCC', 'BAK', 'delta', 'vernalis']
delta_suffix = ['gains', 'gains', 'gains', 'depletions', 'pump', 'pump', 'inflow', 'inflow']
calfews_data.add_extra_stations(delta_labels, delta_suffix)

In [ ]:
#Checks
calfews_data.data_timeseries

In [ ]:
#setup input scenario data file with all key/data type pairs
calfews_data.initialize_dataframe()
calfews_data_m.initialize_dataframe()

In [ ]:
#Checks 
calfews_data_m.real_time_data


In [ ]:
##Monthly Stations Column Lists:
# when CDEC data uses different keys than the main 'basin key'
# we need to associate that key with the main 'basin key' for a specific data type
calfews_data_m.add_station_map(stations_basin, stations_basin, ['inf',])



In [ ]:
#Changing Trinity Monthly Inflows to Clair Engle Lake
calfews_data_m.add_station_map(['TRT',], ['CLE',], ['inf',])

In [ ]:
#Checks
calfews_data_m.real_time_data

In [ ]:
# these are the station keys for monthly full-natural flow associated with each key in stations_basin
# TNL - Trinity River at Lewiston has FNF records going back to 1911
stations_monthly_fnf = ['SBB', 'TNL', 'FTO', 'YRS', 'AMF', 'CSN', 'MKM', 'NHG', 'SNS', 'TLG', 'MRC', 'SJF', 'KGF', 'KWT', 'SCC', 'KRI']
calfews_data_m.add_station_map(stations_basin, stations_monthly_fnf, ['fnf',])



In [ ]:
#Checks
calfews_data_m.station_use

In [ ]:
# these are the station keys for daily downstream incremental flows associated with each key in stations_basin
#None added for Trinity in second position
station_names = ['WLK', 'none', 'GRL', 'MRY', 'none', 'none', 'none', 'none', 'OBB', 'LGN', 'CRS', 'GRF', 'none', 'none', 'none', 'none']
station_types = ['gains',]
calfews_data.add_station_map(stations_basin, station_names, station_types)

In [ ]:
#Checks
calfews_data.station_use

In [ ]:
# different station keys for Trinity
calfews_data.add_station_map(['TRT',], ['TNL'], ['fnf',])
calfews_data.add_station_map(['TRT',], ['CLE'], ['storage',])
calfews_data.add_station_map(['TRT',], ['CLE'], ['inf',])
calfews_data.add_station_map(['TRT',], ['CLE'], ['otf',])
calfews_data.add_station_map(['TRT',], ['CLE'], ['evap',])
calfews_data.add_station_map(['TRT',], ['CLE'], ['precip',])
calfews_data.add_station_map(['TRT',], ['SHA'], ['fci',]) #--- Just temporary

In [ ]:
# different station keys for New Bullards
station_types = ['storage', 'evap', 'precip']
calfews_data.add_station_map(['YRS',], ['BUL'], station_types)
calfews_data.add_station_map(['YRS',], ['flow',], ['otf',])
calfews_data.add_station_map(['YRS',], ['usefnf',], ['inf',])


In [ ]:
#Checks
calfews_data.station_use

In [ ]:
# different station keys for delta basins (not used in CALFEWS)
station_types = ['storage', 'evap', 'precip']
calfews_data.add_station_map(['MHB',], ['none',], station_types)
calfews_data.add_station_map(['PAR',], [['PAR', 'CMN'],], station_types)
station_types = ['inf', 'otf',]
calfews_data.add_station_map(['MHB',], ['flow',], station_types)
calfews_data_m.add_station_map(['MHB',], ['flow',], ['inf',])
calfews_data.add_station_map(['PAR',], ['CMN',], ['otf',])

In [ ]:
#Checks
calfews_data.station_use

In [ ]:
#full-natural-flow keys for other reservoirs
basin_names = ['PAR', 'NHG', 'DNP', 'MIL', 'PFT', 'KWH', 'SUC']
station_names = ['MKM', 'nofnf', 'TLG', 'SJF', 'PNF', 'TRM', 'SCC']
calfews_data.add_station_map(basin_names, station_names, ['fnf',])
#Trinity values are: MUM, HIG,BNK,BFL, RRM, SHM, 


In [ ]:
# snow station mapping (Trinity Values are added)
basin_names = ['SHA', 'TRT', 'ORO', 'YRS', 'FOL']
station_names = [['SLT', 'STM', 'CDP'], ['MUM', 'BNK', 'RRM'],  ['KTL', 'GRZ', 'PLP'], ['KTL', 'GRZ', 'PLP'], ['CAP', 'SIL', 'HYS']]
calfews_data.add_station_map(basin_names, station_names, ['snow',])
basin_names = ['MHB', 'PAR', 'NHG']
station_names = [['HHM', 'BLK'], ['HHM', 'BLK'], ['HHM', 'BLK']]
calfews_data.add_station_map(basin_names, station_names, ['snow',])
basin_names = ['NML', 'DNP', 'EXC', 'MIL']
station_names = [['DDM', 'GNL', 'REL', 'SLM', 'BLD'], ['DAN', 'TUM', 'HRS', 'PDS'], ['STR', 'TNY'], ['VLC', 'AGP', 'CHM', 'HNT']]
calfews_data.add_station_map(basin_names, station_names, ['snow',])
basin_names = ['PFT', 'KWH', 'SUC', 'ISB']
station_names = [['BSH', 'BCB', 'UBC'], ['FRW', 'GNF'], ['FRW', 'GNF'], ['CBT', 'CSV']]
calfews_data.add_station_map(basin_names, station_names, ['snow',])


In [ ]:
##basin-level data
sensor_basin = [82, 8, 15, 76, 23, 74, 45, 41]
labels_basin = ['snow', 'fnf', 'storage', 'inf', 'otf', 'evap', 'precip', 'gains']
url_parts_d = ['https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations=', '&SensorNums=', '&dur_code=D&Start=', '&End=']
url_parts_m = ['https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations=', '&SensorNums=', '&dur_code=M&Start=', '&End=']
url_parts_h = ['https://cdec.water.ca.gov/dynamicapp/req/CSVDataServlet?Stations=', '&SensorNums=', '&dur_code=H&Start=', '&End=']


In [ ]:
url_parts_h

In [ ]:
hourly_dict = {}
for x in stations_basin:
  hourly_dict[x] = []
for x in san_luis_labels:
  hourly_dict[x] = []
for x in delta_labels:
  hourly_dict[x] = []
hourly_dict['MIL'].append('gains')

In [ ]:
#Checks
calfews_data.station_use

In [ ]:
# link and read all data
calfews_data_m.link_api(stations_basin, [76,], ['inf',], url_parts_m, url_parts_h, hourly_dict)
calfews_data_m.link_api(stations_basin, [65,], ['fnf',], url_parts_m, url_parts_h, hourly_dict)
calfews_data.link_api(stations_basin, sensor_basin, labels_basin, url_parts_d, url_parts_h, hourly_dict)
calfews_data.link_api(['PFT', 'KWH', 'ISB'], [64,], ['evap',], url_parts_d, url_parts_h, hourly_dict)
calfews_data.adjust_values('MIL', 'gains', ['MI1', 'MI2'], [85, 85], url_parts_d)
calfews_data_m.find_ratios(stations_basin, 'inf')
calfews_data.fill_missing(stations_basin, calfews_data_m.station_coefs, calfews_data_m.real_time_data)
calfews_data.adjust_fnf_monthly(calfews_data_m.real_time_data, stations_basin)


In [ ]:
#Generate data columns for San Luis, Delta Pumps, and Old-Middle River Flows
san_luis_stations = ['SNL', 'SLF', 'LUS', 'TRP', 'HRO', 'OMR']
sensor_sl  = [15, 15, 15, 70, 70, 41]

# add station maps for san luis, read data
for snl_l, snl_sta, snl_sens, snl_suf in zip(san_luis_labels, san_luis_stations, sensor_sl, san_luis_suffix):
  calfews_data.add_station_map([snl_l,], [snl_sta,], [snl_suf,])
  if snl_l == 'SLS' or snl_l == 'SLF':
    calfews_data.link_api([snl_l,], [snl_sens,], [snl_suf,], url_parts_m, url_parts_h, hourly_dict)
    calfews_data.make_monthly_daily(snl_l, snl_suf)
  else:
    calfews_data.link_api([snl_l,], [snl_sens,], [snl_suf,], url_parts_d, url_parts_h, hourly_dict)

In [ ]:
# read delta station data from dayflow database
calfews_data.link_dayflow_api()

cdec_delta_labels = ['SAC', 'SAC', 'SJ', 'delta', 'BAK']
cdec_delta_stations = ['YBY', 'FPT', 'VNS', 'SFS', 'BKS']
cdec_delta_sensors = [41, 20, 41, 45, 272]
cdec_delta_suffix = ['gains', 'gains', 'gains', 'depletions', 'pump']
for del_l, del_sta, del_sen, del_suf in zip(cdec_delta_labels, cdec_delta_stations, cdec_delta_sensors, cdec_delta_suffix):
  calfews_data.add_station_map([del_l,], [del_sta,], [del_suf,])
  calfews_data.link_api([del_l,], [del_sen,], [del_suf,], url_parts_d, url_parts_h, hourly_dict, use_delta_cats = True)

In [ ]:
calfews_data.assemble_fci(url_parts_d, url_parts_m)



In [ ]:
calfews_data.fix_missing_inputs()
calfews_data.real_time_data.to_csv('cord-sim_realtime.csv')
calfews_data_m.real_time_data.to_csv('cord-sim_realtime_monthly.csv')

In [ ]:
old_filename = 'calfews_src/data/input/calfews_src-data.csv'
new_filename = 'cord-sim_realtime.csv'
calfews_data.make_figure_comparison(stations_basin, labels_basin, old_filename, new_filename)